<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html exercisesweek37.do.txt  -->
<!-- dom:TITLE: Exercises week 37 -->

# Exercises week 37
**Implementing gradient descent for Ridge and ordinary Least Squares Regression**

Date: **September 8-12, 2025**

## Learning goals

After having completed these exercises you will have:
1. Your own code for the implementation of the simplest gradient descent approach applied to ordinary least squares (OLS) and Ridge regression

2. Be able to compare the analytical expressions for OLS and Ridge regression with the gradient descent approach

3. Explore the role of the learning rate in the gradient descent approach and the hyperparameter $\lambda$ in Ridge regression

4. Scale the data properly

## Simple one-dimensional second-order polynomial

We start with a very simple function

$$
f(x)= 2-x+5x^2,
$$

defined for $x\in [-2,2]$. You can add noise if you wish. 

We are going to fit this function with a polynomial ansatz. The easiest thing is to set up a second-order polynomial and see if you can fit the above function.
Feel free to play around with higher-order polynomials.

## Exercise 1, scale your data

Before fitting a regression model, it is good practice to normalize or
standardize the features. This ensures all features are on a
comparable scale, which is especially important when using
regularization. Here we will perform standardization, scaling each
feature to have mean 0 and standard deviation 1.

### 1a)

Compute the mean and standard deviation of each column (feature) in your design/feature matrix $\boldsymbol{X}$.
Subtract the mean and divide by the standard deviation for each feature.

We will also center the target $\boldsymbol{y}$ to mean $0$. Centering $\boldsymbol{y}$
(and each feature) means the model does not require a separate intercept
term, the data is shifted such that the intercept is effectively 0
. (In practice, one could include an intercept in the model and not
penalize it, but here we simplify by centering.)
Choose $n=100$ data points and set up $\boldsymbol{x}$, $\boldsymbol{y}$ and the design matrix $\boldsymbol{X}$.

In [ ]:
from random import random, seed
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
np.random.seed(452)

numb = 1000
X = np.random.rand(numb,1) * 4 - 2
Y = 2.0 - X + 5*(X**2) + np.random.uniform(0, 25)

In [ ]:
# Standardize features (zero mean, unit variance for each feature)
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X_std[X_std == 0] = 1  # safeguard to avoid division by zero for constant features
X_norm = (X - X_mean) / X_std

# Center the target to zero mean (optional, to simplify intercept handling)
y_mean = Y.mean(axis=0)
Y_std = Y.std(axis=0)
print(f'y std is: {Y_std}')
Y_std[Y_std == 0] = 1
y_centered = (Y - y_mean)/Y_std


as we get y quite spreaded from 0 to 25+ with standart deviation 6.12 I desided that we can devide y bu std as well as x values.

In [ ]:
# Create the figure and axes for subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns

# First plot: Scatter plot of original X and Y
axs[0].scatter(X, Y, color='blue', alpha=0.5)
axs[0].set_title('Scatter Plot of Random Points')
axs[0].set_xlabel('X values')
axs[0].set_ylabel('Y values')

# Second plot: Scatter plot of centered X and centered Y
axs[1].scatter(X_norm, y_centered, color='red', alpha=0.5)
axs[1].set_title(' Plot of Normalize Y and X')
axs[1].set_xlabel('X values')
axs[1].set_ylabel('Centered Y values')

# Adjust layout to avoid overlap
plt.tight_layout()
plt.show()


Fill in the necessary details. Do we need to center the $y$-values? 

After this preprocessing, each column of $\boldsymbol{X}_{\mathrm{norm}}$ has mean zero and standard deviation $1$
and $\boldsymbol{y}_{\mathrm{centered}}$ has mean 0. This makes the optimization landscape
nicer and ensures the regularization penalty $\lambda \sum_j
\theta_j^2$ in Ridge regression treats each coefficient fairly (since features are on the
same scale).

<div style="background-color: #f0f0f0; border: 1px solid #d3d3d3; padding: 10px; border-radius: 5px;">

There is no strict need to center y values, but it can help as to get better understanding of data. On the plot above we see that we have x values from -2 to 2, but y can take any value from 0 up to 30. Centering it can help to increase interpretability. 

## Exercise 2, calculate the gradients

Find the gradients for OLS and Ridge regression using the mean-squared error as cost/loss function.

  .

This exercise interprited as theoretical and contains calculations of gradients for OLS and Ridge regression.
Full working code for the same in part 4 of the weekly exercises.

For OLS we have following loss function:
$$
C(\theta) = \frac{1}{n}||X\theta-\mathbf{y}||_{2}^{2} = \frac{1}{n}\sum_{i=1}^{100}\left[ (\theta_0 + \theta_1 x_i)^2 - 2 y_i (\theta_0 + \theta_1 x_i) + y_i^2\right]
$$

$$
\frac{ \partial ( {C(\theta}))}{\partial {\theta_0}}=
2(\theta_0 + \theta_1 x_i) - 2 y_i=
2(\theta_0 + \theta_1 x_i - y_i)
$$

$$
\frac{ \partial ( {C(\theta}))}{\partial {\theta_1}}= 
2(\theta_0 + \theta_1 x_i)x_i - 2 y_i x_i=
2 x_i (\theta_0 + \theta_1 x_i - y_i)
$$

Thus in matrix form we can write gradien as:
$$
\nabla_{\theta} C(\theta) = \frac{2}{n}X^T(X\theta - \mathbf{y}),
$$

and we can approximate to the theta value by subtracting from current theta gradint multiplied on constant rate that can be either choosen randomly, or as 1/ $\lambda$ where $\lambda$ is a max eigenvalue of Hessian matrix 

$$
\theta_{k+1} = \theta_k - \eta \nabla_\theta C(\theta_k)
$$

when the value of $\theta_k$ is relatively small (less than $10^{-8}$) we can stop iterating to save some computing power.

Gradient descend for Ridge regression calculated similarly, but with additional $\lambda\theta^2$ component:

$$
C_{\text{ridge}}(\theta) = \frac{1}{n} \| X \theta - \mathbf{y} \|^2 + \lambda \| \theta \|^2 =
\frac{1}{n} \sum_{i=1}^{n}( \left[ (\theta_0 + \theta_1 x_i)^2 - 2 y_i (\theta_0 + \theta_1 x_i) + y_i^2 \right] + \lambda \| \theta \|^2)
$$


$$
\frac{ \partial ( {C_{ridge}(\theta}))}{\partial {\theta_0}} =
2(\theta_0 + \theta_1 x_i) - 2 y_i + 2 \lambda\theta =
2((\theta_0 + \theta_1 x_i) - y_i + \lambda\theta_0) 
$$

$$
\frac{ \partial ( {C_{ridge}(\theta}))}{\partial {\theta_1}} =
2(\theta_0 + \theta_1 x_i) - 2 y_i + 2 \lambda\theta =
2((\theta_0 + \theta_1 x_i)x_i - y_i x_i + \lambda\theta_1)
$$

$$
\nabla_{\theta} C_{ridge}(\theta) = 2 (\frac{1}{n}X^T(X\theta - \mathbf{y})+\lambda \theta).
$$

This gradient we use the same way as for OLS and stop calculations when the value is arbitrary low.

## Exercise 3

In [ ]:
#split data to test and train
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_centered, test_size=0.2)
x_train = X_train[:, 0] # These are used for plotting
x_test = X_test[:, 0] # These are used for plotting

In [ ]:
# the number of datapoints
n = int(numb * 0.8)
m = int(numb * 0.2)
X = np.c_[np.ones((n,1)), X_train, X_train**2]

X_test = np.c_[np.ones((m,1)), X_test, X_test**2]

In [ ]:
# Set regularization parameter, either a single value or a vector of values
# Note that lambda is a python keyword. The lambda keyword is used to create small, single-expression functions without a formal name. 
#These are often called "anonymous functions" or "lambda functions."
lam = 0.01

# Analytical form for OLS and Ridge solution: theta_Ridge = (X^T X + lambda * I)^{-1} X^T y and theta_OLS = (X^T X)^{-1} X^T y
I = np.eye(len(X.T @ X) )
theta_closed_formRidge = np.linalg.inv(X.T @ X + 2 * lam * I ) @ X.T @ y_train
theta_closed_formOLS = np.linalg.inv(X.T @ X ) @ X.T @ y_train

print("Closed-form Ridge coefficients: \n", theta_closed_formRidge)
print("Closed-form OLS coefficients: \n", theta_closed_formOLS)

This computes the Ridge and OLS regression coefficients directly. The identity
matrix $I$ has the same size as $X^T X$. It adds $\lambda$ to the diagonal of $X^T X$ for Ridge regression. We
then invert this matrix and multiply by $X^T y$. The result
for $\boldsymbol{\theta}$  is a NumPy array of shape (n$\_$features,) containing the
fitted parameters $\boldsymbol{\theta}$.

### 3a)

Finalize, in the above code, the OLS and Ridge regression determination of the optimal parameters $\boldsymbol{\theta}$.

<div style="background-color: #f0f0f0; border: 1px solid #d3d3d3; padding: 10px; border-radius: 5px;">


As long as the lambda value is near 0, we get almost identical values for thetas calculated by Ridge regression and OLS. 

### 3b)

Explore the results as function of different values of the hyperparameter $\lambda$. See for example exercise 4 from week 36.

In [ ]:
# Calculate MSE
MSE_train = mean_squared_error(y_train, X @ theta_closed_formOLS)
MSE_test = mean_squared_error(y_test, X_test @ theta_closed_formOLS)

print("MSE train OSL:", MSE_train)
print("MSE test: OSL", MSE_test)

In [ ]:
# Define the range for lambda
lamb = np.arange(0, 6000, 1000)
degrees = len(lamb)  # The number of degrees will match the number of lambdas
MSE_train_R = np.zeros(degrees)
MSE_test_R = np.zeros(degrees)

for i in range(degrees):
    theta = np.linalg.inv(X.T @ X + 2 * lamb[i] * I ) @ X.T @ y_train
    # Calculate MSE
    MSE_train_R[i] = mean_squared_error(y_train, X @ theta)
    MSE_test_R[i] = mean_squared_error(y_test, X_test @ theta)

# Print headers
print(f"{'Lambda':<20} {'MSE Train':<20} {'MSE Test':<20}")
for i in range(degrees):
    print(f"{lamb[i]:<20.8f} {MSE_train_R[i]:<20.8f} {MSE_test_R[i]:<20.8f}")

In [ ]:
# Plotting the results
plt.figure(figsize=(10, 5))
plt.plot(lamb, MSE_train_R, marker='o', label='MSE Train Ridge', color='blue')
plt.plot(lamb, MSE_test_R, marker='o', label='MSE Test Ridge', color='red')
plt.xscale('log')  # Set x-axis to logarithmic scale
plt.title('Mean Squared Error for Training and Testing Ridge regression')
plt.xlabel('Lambda (log scale)')
plt.ylabel('Mean Squared Error')
plt.xticks(rotation=45)  # Optionally adjust tick labels for clarity
plt.legend()
plt.grid()
plt.show()

We see that both train and test errors increase as we move away from close to 0 value. 

## Exercise 4, Implementing the simplest form for gradient descent

Alternatively, we can fit the ridge regression model using gradient
descent. This is useful to visualize the iterative convergence and is
necessary if $n$ and $p$ are so large that the closed-form might be
too slow or memory-intensive. We derive the gradients from the cost
functions defined above. Use the gradients of the Ridge and OLS cost functions with respect to
the parameters  $\boldsymbol{\theta}$ and set up (using the template below) your own gradient descent code for OLS and Ridge regression.

Below is a template code for gradient descent implementation of ridge:

In [ ]:
# Gradient descent parameters, learning rate eta first
eta = 0.1
# Then number of iterations
num_iters = 1000

# Initialize weights for gradient descent
theta = np.zeros(n_features)

# Gradient descent loop
for t in range(num_iters):
    # Compute gradients for OSL and Ridge
    grad_OLS = ?
    grad_Ridge = ?
    # Update parameters theta
    theta_gdOLS = ?
    theta_gdRidge = ? 

# After the loop, theta contains the fitted coefficients
theta_gdOLS = ?
theta_gdRidge = ?
print("Gradient Descent OLS coefficients:", theta_gdOLS)
print("Gradient Descent Ridge coefficients:", theta_gdRidge)

### 4a)

Write first a gradient descent code for OLS only using the above template.
Discuss the results as function of the learning rate parameters and the number of iterations

In [ ]:
# Hessian matrix
H = (2.0/n)* X.T @ X
EigValues, EigVectors = np.linalg.eig(H)


theta_gdOLS =np.zeros((3, 1))# start theta as random var.
eta = 1.0/np.max(EigValues) 
Niterations = 1000
convrg = 0

for iter in range(Niterations):
    gradient = (2.0/n)*X.T @ (X @ theta_gdOLS-y_train)
    old_theta = theta_gdOLS.copy()
    theta_gdOLS -= eta*gradient
    convrg +=1
    if np.all(gradient < 1e-8):
        break

print("Gradient Descent OLS coefficients:",'\n',theta_gdOLS)
print('Converges at step: ',convrg)

### 4b)

Write then a similar code for Ridge regression using the above template.
Try to add a stopping parameter as function of the number iterations and the difference between the new and old $\theta$ values. How would you define a stopping criterion?

In [ ]:
#Ridge
XT_X = X.T @ X
#Ridge parameter lambda
lmbda  = 0.001

# Hessian matrix
H = (2.0/n)* XT_X +2*lmbda* np.eye(XT_X.shape[0])
# Get the eigenvalues
EigValues, EigVectors = np.linalg.eig(H)
print(f"Eigenvalues of Hessian Matrix: \n {EigValues}")


# Start plain gradient descent
theta = np.zeros((3, 1))

eta = 1.0/np.max(EigValues)
Niterations = 1000

converg = 0
for iter in range(Niterations):
    gradients = (2.0/n)*X.T @ (X @ theta-y_train)+2*lmbda*theta
    theta -= eta*gradients
    converg +=1
    if np.all(gradients < 1e-8):
        break


print('Theta values:','\n',theta)
print('Converges at step: ',converg)

### Comment
The smaller the lambda values we choose, the closer it approximates to analytical and OLS values.

Both Ridge and OSL have a comparable number of iterations with the same parameter. Once gradient becomes less than $10^{-8}$ we stop our calculations as they become good enough. 

## Exercise 5, Ridge regression and a new Synthetic Dataset

We create a synthetic linear regression dataset with a sparse
underlying relationship. This means we have many features but only a
few of them actually contribute to the target. In our example, we’ll
use 10 features with only 3 non-zero weights in the true model. This
way, the target is generated as a linear combination of a few features
(with known coefficients) plus some random noise. The steps we include are:

Decide on the number of samples and features (e.g. 100 samples, 10 features).
Define the **true** coefficient vector with mostly zeros (for sparsity). For example, we set $\hat{\boldsymbol{\theta}} = [5.0, -3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0]$, meaning only features 0, 1, and 6 have a real effect on y.

Then we sample feature values for $\boldsymbol{X}$ randomly (e.g. from a normal distribution). We use a normal distribution so features are roughly centered around 0.
Then we compute the target values $y$ using the linear combination $\boldsymbol{X}\hat{\boldsymbol{\theta}}$ and add some noise (to simulate measurement error or unexplained variance).

Below is the code to generate the dataset:

In [ ]:
import numpy as np

# Set random seed for reproducibility
np.random.seed(0)

# Define dataset size
n_samples = 100
n_features = 10

# Define true coefficients (sparse linear relationship)
theta_true = np.array([5.0, -3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0])

# Generate feature matrix X (n_samples x n_features) with random values
X = np.random.randn(n_samples, n_features)  # standard normal distribution

# Generate target values y with a linear combination of X and theta_true, plus noise
noise = 0.5 * np.random.randn(n_samples)    # Gaussian noise
y = np.expand_dims(X @ theta_true + noise, axis=1) # y missing dimention


This code produces a dataset where only features 0, 1, and 6
significantly influence $\boldsymbol{y}$. The rest of the features have zero true
coefficient. For example, feature 0 has
a true weight of 5.0, feature 1 has -3.0, and feature 6 has 2.0, so
the expected relationship is:

$$
y \approx 5 \times x_0 \;-\; 3 \times x_1 \;+\; 2 \times x_6 \;+\; \text{noise}.
$$

You can remove the noise if you wish to. 

Try to fit the above data set using OLS and Ridge regression with the analytical expressions and your own gradient descent codes.

If everything worked correctly, the learned coefficients should be
close to the true values [5.0, -3.0, 0.0, …, 2.0, …] that we used to
generate the data. Keep in mind that due to regularization and noise,
the learned values will not exactly equal the true ones, but they
should be in the same ballpark.  Which method (OLS or Ridge) gives the best results?

## Coment:
i tried to standardize x value, but it leads to increase in MSE in both test and train data

In [ ]:
# Standardize features (zero mean, unit variance for each feature)
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X_std[X_std == 0] = 1  # safeguard to avoid division by zero for constant features
X_norm = (X - X_mean) / X_std


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

Analytical:

In [ ]:
I = np.eye(len(X_train.T @ X_train) )
theta_closed_formRidge = np.linalg.inv(X_train.T @ X_train + 2 * lam * I ) @ X_train.T @ Y_train
theta_closed_formOLS = np.linalg.inv(X_train.T @ X_train ) @ X_train.T @ Y_train

print("Closed-form Ridge coefficients: \n", np.round(theta_closed_formRidge, 0))
print("Closed-form OLS coefficients: \n", np.round(theta_closed_formOLS, 0))

### MSE for train and test data analytical solution

In [ ]:
#OLS
MSE_train_OLS = mean_squared_error(Y_train, X_train @ theta_closed_formOLS)
MSE_test_OLS = mean_squared_error(Y_test, X_test @ theta_closed_formOLS)

print("MSE for OLS train data: \n", MSE_train_OLS)
print("MSE for OLS test data:: \n", MSE_test_OLS, '\n')

#Ridge regression
MSE_train_R = mean_squared_error(Y_train, X_train @ theta_closed_formRidge)
MSE_test_R = mean_squared_error(Y_test, X_test @ theta_closed_formRidge)

print("MSE for analytical Ridge regression train data: \n", MSE_train_R)
print("MSE for analytical Ridge regression test data:: \n", MSE_test_R)

### Aproximation for OLS

In [ ]:
# Hessian matrix
H_OLS = (2.0/n_samples)* X_train.T @ X_train
EigValues, EigVectors = np.linalg.eig(H_OLS)

theta_gdOLS =np.zeros((n_features, 1))# start theta as random var.
eta = 1.0/np.max(EigValues) 
Niterations = 1000
convrg = 0

for iter in range(Niterations):
    gradient = (2.0/n_samples)*X_train.T @ (X_train @ theta_gdOLS-Y_train)
    old_theta = theta_gdOLS.copy()
    theta_gdOLS -= eta*gradient
    convrg +=1
    if np.all(gradient < 1e-8):
        break

print("Gradient Descent OLS coefficients:",'\n',np.round(theta_gdOLS,0))
print('Converges at step: ',convrg)

### Aproximation for Ridge

In [ ]:
#Ridge
XT_X = X_train.T @ X_train
#Ridge parameter lambda
lmbda  = 0.01

# Hessian matrix
H_Ridge = (2.0/n_samples)* XT_X +2*lmbda* np.eye(XT_X.shape[0])
# Get the eigenvalues
EigValues, EigVectors = np.linalg.eig(H_Ridge)

# Start plain gradient descent
theta_gdRidge =np.zeros((n_features, 1))

eta = 1.0/np.max(EigValues)
Niterations = 1000

converg = 0
for iter in range(Niterations):
    gradients = (2.0/n_samples)*X_train.T @ (X_train @ theta_gdRidge-Y_train)+2*lmbda*theta_gdRidge
    theta_gdRidge -= eta*gradients
    converg +=1
    if np.all(gradients < 1e-8):
        break

print('Theta values:','\n',np.round(theta_gdRidge,0))
print('Converges at step: ',converg)

### MSE for train and test data gradient

In [ ]:
#OLS
MSE_train_OLS = mean_squared_error(Y_train, X_train @ theta_gdOLS)
MSE_test_OLS = mean_squared_error(Y_test, X_test @ theta_gdOLS)

print("MSE for OLS train data: \n", MSE_train_OLS)
print("MSE for OLS test data:: \n", MSE_test_OLS, '\n')

#Ridge
MSE_train_Ridge = mean_squared_error(Y_train, X_train @ theta_gdRidge)
MSE_test_Ridge = mean_squared_error(Y_test, X_test @ theta_gdRidge)

print("MSE for OLS train data: \n", MSE_train_Ridge)
print("MSE for OLS test data:: \n", MSE_test_Ridge, '\n')

Which method (OLS or Ridge) gives the best results?

When lambda value is at 0.001 we getting comparable accuracy results from both OSL and Ridge, but they are techically wery close to each other. Once we increase lambda, the prediction accuracy of Ridge regression model is declining. Thus for our data the OSL model is better fit that provides better accuracy.